In [1]:
import os

from getpass import getpass
import nest_asyncio

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

True

In [2]:
CO_API_KEY = os.environ['CO_API_KEY']
GEMINI_API_KEY = os.environ['GEMINI_API_KEY'] 
QDRANT_URL = os.environ['QDRANT_URL'] 
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] 

In [4]:
from pathlib import Path

def create_directory(directory_name):
    path = Path(directory_name)
    path.mkdir(parents=True, exist_ok=True)
    print(f"Directory '{directory_name}' created successfully.")

create_directory("../data_text")

Directory '../data_text' created successfully.


In [8]:
import requests

def load_text_from_url(url: str) -> str:
    """
    Fetches and returns the text content from the specified URL.

    Parameters:
    - url: The URL of the text file to fetch.

    Returns:
    - The text content of the file if the request is successful; otherwise, an error message.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # This will raise an HTTPError if the response was an error
        return response.text
    except requests.RequestException as e:
        return f"Failed to load content from {url}. Error: {e}"

url = "https://www.gutenberg.org/files/10763/10763.txt"

text_content = load_text_from_url(url)

# 🗄️ Storing

Loading and indexing data costs time and money.

By default, indexed data is stored in memory. But, you can store your data to avoid the time and costs associated with re-indexing them.  The simplest way to do this **persisting to disk**.

Each `Index` object has a `.persist()` method, which will write all the data to disk at the specified location.

Now that we've dowloaded data, let's:

1) Load as Document
2) Parse as Nodes
3) Create index

In [9]:
from llama_index.core import SimpleDirectoryReader

file_path = "../data_text/pg10763.txt"

document = SimpleDirectoryReader(input_files=[file_path], filename_as_id=True).load_data()

In [10]:
# Create Node parser
from llama_index.core.node_parser import SentenceSplitter

sentence_splitter = SentenceSplitter(
    chunk_size=512, 
    chunk_overlap=16,
    paragraph_separator="\n\n\n\n" 
)

In [29]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

In [30]:
embed_model = genai.embed_content(model="models/text-embedding-004", content="Sirah Nabi")

In [34]:
from llama_cloud import GeminiEmbedding


model_name = "models/embedding-001"

embed_model = GeminiEmbedding(
    model_name=model_name, api_key=GEMINI_API_KEY, title="this is a document"
)

# ☁️ Using a Vector Database

We'll use qdrant as our vector database of choice throughout this course.

To use qdrant to store embeddings from the `VectorStoreIndex`, you need to:

- Initialize the qdrant client

- Create a `Collection` to store your data in qdrant

- Assign qdrant as the `vector_store` in a `StorageContext`

- Initialize your `VectorStoreIndex` using that `StorageContext`

Below, we initialize a `QdrantClient` for interacting with qdrant, an open-source vector store. 


In [35]:
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore

# initialize qdrant client
client = qdrant_client.QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

vector_store = QdrantVectorStore(
    client=client, 
    collection_name="it_can_be_done",
    embed_model=embed_model,
)

# 🗃️ Storage Context

`StorageContext` in `LlamaIndex` is a core abstraction that revolves around the storage of `Nodes`, indices, and vectors.  It facilitates data storage and retrieval.

It is a utility container that supports the following:

 - `docstore`: A [`BaseDocumentStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/storage/docstore/types.py) for storing nodes.

 - `index_store`: A [`BaseIndexStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/storage/index_store/types.py#L13) for storing indices.

 - `vector_store`: A [`VectorStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/vector_stores/simple.py) for storing vectors.

 - `graph_store`: A [`GraphStore`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/graph_stores/simple.py) for storing knowledge graphs.

Below we instantiate the `StorageContext` from default settings indicating that we want to use a vector store.

In [38]:
from llama_index.core import StorageContext

# assign qdrant vector store to storage context
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    )

In [41]:
from llama_index.core import  VectorStoreIndex

# create the index
index = VectorStoreIndex.from_documents(
    document,  
    store_nodes_override=True,
    transformation=[sentence_splitter],  # Make sure this is defined correctly
    embed_model=embed_model,
    storage_context=storage_context,
)

ValueError: "GeminiEmbedding" object has no field "callback_manager"

# 🪃 Retrieval

A `Retriever` is an interface exposed by the `Index`. An `Index` with its `Retriever` is used for storing and fetching data. The `Retriever` is a part of the `Index` and is used to retrieve the data stored in the Index.


### LlamaIndex provides [many different types of retrievers](https://github.com/run-llama/llama_index/tree/main/llama-index-core/llama_index/core/retrievers) to fetch relevant information from ingested data based on a given query. 

Some examples include

### Vector Retriever

The vector retriever uses vector similarity search to find the most relevant nodes (chunks of text) based on the query embedding. It requires a vector database like to store and search through the node embeddings.

### [Fusion Retriever](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/retrievers/fusion_retriever.py)

The fusion retriever generates multiple queries from the original query, performs retrieval over an ensemble of retrievers for each query, and then fuses and reranks the results across all queries. This aims to better capture the query intent through query rewriting and ensembling.

### [Recursive Retriever](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/retrievers/recursive_retriever.py)

The recursive retriever allows for hierarchical retrieval by first retrieving coarse nodes and then recursively retrieving finer-grained nodes within those coarse nodes. This can be useful for multi-level indexing and retrieval.

You can also combine retrievers in interesting ways and build out more advanced retrieval strategies, as we will see later in this course.


### In the example here, we're using a Vector Retriever

 - 🔍 When searching, your query is also converted into a vector embedding. 
 
- 🗂️ The `VectorStoreIndex` then performs a mathematical operation to rank embeddings based on semantic similarity to your query.

- 🔝 Top-k semantic retrieval is the simplest wasy to query a vector index.

- ⩬ You can also apply a similarity threshold  (e.g., only return results that are more similar than some value)


In [40]:
retirever = index.as_retriever(
    similarity_top_k=5,
    similarity_threshold=0.75)

NameError: name 'index' is not defined